In [ ]:
%matplotlib inline

import os
import pickle
import numpy as np

import gym
import neat
import cv2

import utils.PyPlotReporter
import utils.CSVReporter
import utils.SubstrateNet as Substrate
from utils.EnvEvaluator import * 
from utils.SubstrateNet import GridMappingBounds, DirectMapping
from utils.Profiling import get_exec_times_pd
from utils.Atari import CONTROLLER_TO_ACTION,CONTROLLER_TO_ACTION_SHORT

In [ ]:
class HyperNeatEnvEvaluator(HyperNeatEnvEvaluator):
    
    def make_substrate(self):
        sub = Substrate.Sequential(min_abs_weight=0.1,bias=False)
        sub.input(Substrate.Input(GridMappingBounds((42,32), 0, range_x=(-1,1), range_y=(-1, 2))))
#        sub.add(Substrate.Dense(grid_mapping_bounds((21,16), 1, range_x=(-0.5,0.5), range_y=(-0.5, 0.5))))
        #                       output coords: FIRE , UP      , RIGHT, LEFT     , DOWN        , z-coord       
        sub.add(Substrate.Dense(DirectMapping([(1,0), (-0.5,1), (0,0), (-1, 0), (-0.5,-1)], 1)))
        sub.compile()
        return sub

    def preproc_img(self, observation):
        grey = cv2.cvtColor(observation, cv2.COLOR_RGB2GRAY)
        grey = cv2.resize(grey,dsize=(32,42), interpolation=cv2.INTER_AREA) 
       # grey = (grey - np.mean(grey)) / np.std(grey) 
        grey = grey / 255
        return grey
    
    def activate_net(self, sub, observation): 
        grey = self.preproc_img(observation)
        activation = sub.activate(grey)[0]
        out = tuple(activation > .5)
        action = CONTROLLER_TO_ACTION_SHORT[out]
        return action

evaluator = HyperNeatEnvEvaluator("DemonAttack-v0", 2000, n_workers=8, seed=None)
#evaluator = HyperNeatEnvEvaluator("ChopperCommand-v0", 1000, n_workers=8, seed=1111)

In [ ]:
config_path = "./configs/hyperatari.cfg" 

config = neat.Config(
    neat.DefaultGenome,
    neat.DefaultReproduction,
    neat.DefaultSpeciesSet,
    neat.DefaultStagnation,
    config_path,
)

In [ ]:
pop = neat.Population(config)
pop.add_reporter(utils.CSVReporter.CSVReporter("saves/progress.csv"))
pop.add_reporter(utils.PyPlotReporter.PyPlotReporter())
pop.add_reporter(neat.Checkpointer(1, None, "saves/cp_atari_6_"))

In [ ]:
gnome = pop.run(evaluator.eval_all_genomes, 3)

In [ ]:
# print profiling information
get_exec_times_pd()

In [ ]:
# plot progress from csv file
utils.CSVReporter.CSVReporter.plot("saves/progress.csv")

In [ ]:
# reload from checkpoint and find best gnome
pop = neat.Checkpointer.restore_checkpoint("saves/cp_atari_6_2")
pop.add_reporter(neat.StdOutReporter(False))
gnome = pop.run(evaluator.eval_all_genomes, 1)

## Visualization

In [ ]:
import utils.Visualization

In [ ]:
evaluator.show(gnome, config, delay=0.01, random=True)

In [ ]:
utils.Visualization.create_video(evaluator,gnome, config, fps=120)